In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import bokeh
from bokeh.plotting import figure, output_file, show, ColumnDataSource
import bokeh.io
import bokeh.models
from bokeh.models import HoverTool
import bokeh.palettes
import bokeh.plotting
import random
from random import sample

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam

Using TensorFlow backend.


In [14]:
# Preprocessing, normalization done in excel (Xnew = (X-mean)/std))
Data = pd.read_csv('songs4.csv')
Data = Data.iloc[:, 0:18]
#Data = Data.drop(Data[(Data.time_signature > 5)].index)
Data.head()

,ID,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,zero_ts,one_ts,three_ts,four_ts,five_ts
0,5CQ30WqJwcep0pYcV4AMNc,-1.268543,-0.885605,1.050276,-0.280775,-1.258128,-0.630077,0.678645,-0.649848,-0.480625,-1.032112,-1.205528,1.347462,-0.040332,-0.105962,-0.314267,0.378598,-0.152588
1,0yE62JSC9J9C4Q4OBMwBgA,-0.385007,-0.303138,1.050276,0.009403,0.794818,2.212337,-0.630163,-0.658555,-0.504163,-0.175880,-0.772431,-0.113739,-0.040332,-0.105962,3.181954,-2.641279,-0.152588
2,66v8ihTuXUDKqienUaFzIe,-1.191040,-0.723399,-0.901086,-0.765659,0.794818,0.067502,-0.499560,-0.658546,-0.357049,-0.255795,1.629569,-0.519527,-0.040332,-0.105962,-0.314267,0.378598,-0.152588
3,6l4HJkk0CwELcf1CgrbAth,1.511236,0.098691,-1.179852,0.430741,0.794818,-0.338460,-0.052175,-0.658555,0.113717,-0.000828,0.370186,-0.701656,-0.040332,-0.105962,-0.314267,0.378598,-0.152588
4,68rPZWbtaNGUh15AVTSlNn,-0.152498,0.330941,-1.179852,0.428633,0.794818,0.561423,-0.893036,-0.658555,-0.527702,-0.274822,0.025924,0.156217,-0.040332,-0.105962,-0.314267,0.378598,-0.152588


In [ ]:
# check number of rows
Data.count()

In [ ]:
# Divide into testing and training
x = Data.drop('valence', 1)
y = Data.valence
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20)
train = xtrain
train['valence'] = ytrain
train = train [0: 5000]
train.head()

In [ ]:
# Create correlation matrix
M = train.corr()

In [ ]:
M

In [ ]:
plt.imshow(M)
plt.colorbar()
plt.show()

In [ ]:
U, E, VT = np.linalg.svd(M)

In [ ]:
plt.plot(E)
plt.show()

In [ ]:
P = np.dot(U[:,:2],np.diag(E[:2]))

In [ ]:
plt.plot(P[:,0], P[:,1],'o')
plt.show()

In [ ]:
N = train.T
N.columns = N.iloc[0]
N = N.drop('ID')
N = N.astype(float)

In [ ]:
# Takes 10-15 minutes
N = N.corr()
N.head()

In [ ]:
plt.imshow(N)
plt.colorbar()
plt.show()

In [ ]:
U, E, VT = np.linalg.svd(N)

In [ ]:
# Most of the variance can be explained using the first 8 or so components
plt.plot(E[:10])
plt.show()

In [ ]:
P = np.dot(U[:,:2],np.diag(E[:2]))
print(P)

In [17]:
_tools_to_show = 'box_zoom,pan,save,hover,resize,reset,tap,wheel_zoom'        
p = figure(plot_width=400, plot_height=400, title=None, tools=_tools_to_show)

# add a circle renderer with a size, color, and alpha
p.circle(P[:,0], P[:,1], size=4, color="navy", alpha=0.2)

# show the results
show(p)


ValueError: unexpected tool name 'resize', similar tools are reset

In [ ]:
# Random Forest
clf = RandomForestClassifier()
parameters_rand = {
    "n_estimators": sp_randint(300, 2000),
    "max_depth": [3, None],
    "max_features": sp_randint(1, 11),
    "min_samples_split": sp_randint(2, 11),
    "min_samples_leaf": sp_randint(1, 11),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"]
}

n_iter_search = 40
random_search = RandomizedSearchCV(clf, param_distributions=parameters_rand,
                                   n_iter=n_iter_search,
                                   n_jobs=-1)

random_search.fit(xtrain, ytrain)

predicted = random_search.predict(xtest)

print("Accuracy: {}".format(np.mean(predicted == ytest)))

In [21]:
input_shape = xtrain.shape[1]
ae = Sequential()
ae.add(Dense(512,  activation='relu', input_shape=(input_shape,)))
ae.add(Dense(128,  activation='relu'))
ae.add(Dense(1,    activation='linear', name="bottleneck"))
ae.add(Dense(128,  activation='relu'))
ae.add(Dense(512,  activation='relu'))
ae.add(Dense(input_shape,  activation='sigmoid', name="out_layer"))
ae.compile(loss='mean_squared_error',
           optimizer='adadelta',
           metrics=['accuracy'])




In [23]:
## AutoEncoder
history = ae.fit(xtrain, xtrain,
                 batch_size=128,
                 epochs=5,
                 verbose=1,
                 validation_data=(xtest, xtest))


encoder = Model(ae.input, ae.get_layer('bottleneck').output)

encoder.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy'])

encoder.fit(xtrain, ytrain,
            nb_epoch=10,
            batch_size=128,
            shuffle=True,
            validation_data=(xtest, ytest))

print("Autoencoder")
scores = encoder.evaluate(xtest, ytest, verbose=1)
print("Accuracy: ", scores[1])


ValueError: Error when checking input: expected dense_5_input to have shape (18,) but got array with shape (17,)